In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot     
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences       
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

from tensorflow.keras.datasets import imdb  


In [2]:
#load the IMDB dataset word index
imdb_word_index = imdb.get_word_index() 
# Reverse the word index to get words from indices
reverse_word_index = dict([(value, key) for (key, value) in imdb_word_index.items()])               


In [ ]:
#load the model
model = tf.keras.models.load_model('simple_rnn_model.h5')  # Load the saved model   
model.summary()  # Print the model summary  

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_8 (SimpleRNN)        │ (None, 100, 128)       │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,325,315 (5.06 MB)

 Trainable params: 1,325,313 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
model.get_weights()  # Get the weights of the model     

[array([[-0.04477138, -0.09920917,  0.03527232, ...,  0.00158223,
         -0.00085148, -0.00693462],
        [-0.07991624,  0.03256688, -0.07484964, ...,  0.034887  ,
         -0.03252967,  0.02118639],
        [-0.00712351,  0.06910551,  0.00534856, ...,  0.01510307,
         -0.04169187,  0.04203723],
        ...,
        [-0.01755012,  0.00631404, -0.00733489, ..., -0.04336466,
          0.06259421,  0.04185373],
        [-0.07674392,  0.02167653,  0.03881739, ...,  0.02083483,
         -0.04149803, -0.02983357],
        [ 0.09181703, -0.10711601,  0.08972591, ..., -0.02157585,
         -0.0299427 , -0.02686002]], dtype=float32),
 array([[-0.02219686,  0.01742627, -0.19743006, ...,  0.11566348,
          0.00587422, -0.0016385 ],
        [-0.11644522, -0.08310608, -0.12336759, ...,  0.0215508 ,
         -0.12590653, -0.08051443],
        [-0.13699555, -0.15853612, -0.1870735 , ...,  0.08610865,
         -0.1605547 , -0.13153422],
        ...,
        [-0.03981204,  0.13208586,  0.1

In [8]:
#Helper function to decode reviews
def decode_review(encoded_review):  
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])
    return decoded_review   


def preprocess_review(review, maxlen=100):
    # Tokenize the review
    tokenizer = Tokenizer(num_words=5000)  # Use the same num_words as during training
    tokenizer.fit_on_texts([review])  # Fit on the single review
    encoded_review = tokenizer.texts_to_sequences([review])[0]  # Convert to sequence
    padded_review = pad_sequences([encoded_review], maxlen=maxlen, padding='pre')  # Pad the sequence
    return padded_review



In [13]:
#predict on a single review
review = "This movie was fantastic! I loved the plot and the acting was superb."        
padded_review = preprocess_review(review)  # Preprocess the review
prediction = model.predict(padded_review)  # Predict the sentiment  
print(f"Review: {review}")
print("sentiment", "positive" if prediction[0][0] > 0.5 else "negative")  # Determine sentiment based on prediction score           
print(f"Encoded Review: {padded_review}")   
print(f"Prediction: {prediction[0][0]:.4f}")  # Print the prediction score  
decoded_review = decode_review(padded_review[0])  # Decode the review
print(f"Decoded Review: {decoded_review}")  # Print the decoded review  


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Review: This movie was fantastic! I loved the plot and the acting was superb.
sentiment negative
Encoded Review: [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  4  1  5  6  7  2  8  9
   2 10  1 11]]
Prediction: 0.4953
Decoded Review: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? the ? and a of ? to is ? br ? in


In [10]:
#predict on multiple reviews

reviews = [
    "This movie was fantastic! I loved the plot and the acting was superb.",                
    "The film was a complete waste of time. I didn't enjoy it at all.",
    "An average movie with some good moments but overall disappointing.",       
    "I was blown away by the performances and the cinematography.",
    "The story was weak, and the characters were not well developed.",      
    "A masterpiece! The direction and screenplay were top-notch.",
    "I found the movie boring and predictable.",    
    "The soundtrack was amazing, but the plot was lacking.",    
    "A thrilling experience from start to finish!",
    "I wouldn't recommend this movie to anyone. It was terrible."   
]       

# Preprocess the reviews
padded_reviews = [preprocess_review(review) for review in reviews]  # Preprocess each review
padded_reviews = np.vstack(padded_reviews)  # Stack the padded reviews into a single        
# array         
# Predict the sentiment for each review
predictions = model.predict(padded_reviews)  # Predict the sentiment for all reviews    
# Print the predictions
for i, review in enumerate(reviews):    
    print(f"Review {i+1}: {review}")
    print(f"Encoded Review: {padded_reviews[i]}")
    print(f"Prediction: {predictions[i][0]:.4f}")  # Print the prediction score
    decoded_review = decode_review(padded_reviews[i])  # Decode the review
    print(f"Decoded Review: {decoded_review}\n")  # Print the decoded review            


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Review 1: This movie was fantastic! I loved the plot and the acting was superb.
Encoded Review: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  4  1  5  6  7  2  8  9
  2 10  1 11]
Prediction: 0.4953
Decoded Review: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? the ? and a of ? to is ? br ? in

Review 2: The film was a complete waste of time. I didn't enjoy it at all.
Encoded Review: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  